# 유동인구 시간대 및 분기별 분리
---
## 개요
원본 지하철 승하차 데이터에서 하차하는 인원을 대상으로, 분기+역명+요일을 기준으로 하여 총 합산

## 사용 데이터
서울교통공사_역별 일별 시간대별 승하차인원_20211231.csv, 서울특별시 열린 데이터 광장, 서율교통공사 제공

## 구동 시퀸스
- 승하차 데이터에서 하차하는 행만 별도로 추출
- 모든 요일이 있기 때문에, 이를 분기 기준 월에 맞추어서 4개의 데이터 프레임으로 분리
    - 각 데이터 프레임은 1,2,3,4 분기만 해당됨
- 분기별로 나누어진 데이터 프레임에 기준 시간대(00~11시, 11~17시, 17~24시) 결과를 합산하여 컬럼 추가
- 날짜 데이터를 연,월,일,요일의 4개의 데이터로 분리하여, 컬럼 추가
- 역명과 요일을 기준으로 데이터를 그룹화하고, 시간대 데이터만 추출하여 별도 파일 저장

## 주의사항
이 과정은 21년 데이터를 기준으로 작성하였고, 데이터셋마다 컬럼 순서나 명칭등이 살짝 차이나는 점을 반영하지 못했음
- 따라서, 다른 데이터셋에 사용할 때는 해당 함수나 컬럼을 확인하여 변경해야 함

## 기본 모듈 불러오기

In [11]:
import numpy as np
import pandas as pd

import datetime

# 함수 정의

In [36]:
# csv 파일 불러오기, 경로명과 숫자를 받음
# 여기서는 사용하지 않은 상태이나, 활용은 가능
def csv_reader(path, num):
    csv_file = path + str(num) + '.csv'  # 경로명과 순번
    print(csv_file, "reading... please wait")  # 로딩 확인용
    
    data = pd.read_csv(csv_file, encoding="EUC-KR", header=0, index_col=0)
    
    return data


# 빈 데이터프레임 생성 함수
def initialize_df():
    matching_df = pd.DataFrame()

    return matching_df


# 하차 데이터만 별도 분리하는 함수
def arrive_matching(df, data):
    for i in range(len(data)):
        if data.iloc[i]['구분'] == '하차':  # loc를 사용하려면 .loc[i, '구분'] 으로 변경하면 됨
            target_data = pd.DataFrame(data.iloc[i]).T  # Series로 만들어지기 때문에, 인덱스/컬럼의 위치를 변경하는 T 사용
            df = pd.concat([df, target_data], ignore_index=True, axis=0)
    
    return df


# 분기별 분리 함수
def quarter_matching(df):
    qt1, qt2, qt3, qt4 = initialize_df(), initialize_df(), initialize_df(), initialize_df()  # 4개의 분기이기 때문에, 4개를 생성
    # qt1 = qt2 = qt3 = qt4 = initialize_df() 해도 될 것 같음
    
    for i in range(len(df)):
        target_data = pd.DataFrame(df.iloc[i]).T
        date = datetime.datetime.strptime(df.iloc[i]['날짜'], "%Y-%m-%d")  # datetime 모듈을 사용, 날짜 컬럼의 데이터를 입력받아 날짜 데이터 인식
        if date.month in [1,2,3]:  # 1분기
            qt1 = pd.concat([qt1, target_data], axis=0, ignore_index=True)
        elif date.month in [4,5,6]:  # 2분기
            qt2 = pd.concat([qt2, target_data], axis=0, ignore_index=True)
        elif date.month in [7,8,9]:  # 3분기
            qt3 = pd.concat([qt3, target_data], axis=0, ignore_index=True)
        elif date.month in [10,11,12]:  # 4분기
            qt4 = pd.concat([qt4, target_data], axis=0, ignore_index=True)
    
    return qt1, qt2, qt3, qt4


# 시간대 합산 함수
def timezone_summation(df):
# 컬럼명들
# ,날짜,호선,역번호,역명,구분,  : idx 0 ~ 5
# 06시 이전,06시-07시,07시-08시,08시-09시,09시-10시,10시-11시,  : idx 5 ~ 11
# 11시-12시,12시-13시,13시-14시,14시-15시,15시-16시,16시-17시,  : idx 11 ~ 17
# 17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시 이후, : idx 17 ~ 24
# 합 계
    new_df = df.copy()

    # 시간대별 컬럼 추가
    new_df['00시11시'] = None
    new_df['11시17시'] = None
    new_df['17시24시'] = None
    # 추가된 순서와 그 idx는 00시11시(-3), 11시17시(-2), 17시24시(-1)

    for i in range(len(new_df)):
        new_df.iloc[i, -3] = new_df.iloc[i][5:11].apply('sum')  # 00시11시에 해당 시간대 합산하여 추가
        new_df.iloc[i, -2] = new_df.iloc[i][11:17].apply('sum')  # 11시17시에 해당 시간대 합산 추가
        new_df.iloc[i, -1] = new_df.iloc[i][17:24].apply('sum')  # 17시24시에 해당 시간대 합산 추가
    
    return new_df  # 추가된 데이터를 가진 데이터프레임 반환


# 연,월,일,요일 분리 함수
def ymd_split(df):
    
    new_df = df.copy()
    
    # 신규 컬럼 추가 및 요일 dict 추가
    new_df['연'] = new_df['월'] = new_df['일'] = new_df['요일'] = None  # 연,월,일,요일 컬럼 추가
    weekday_dict = {0:'월',1:'화',2:'수',3:'목',4:'금',5:'토',6:'일'}
    
    # 각 컬럼에 맞는 데이터 입력
    for i in range(len(new_df)):
        date = datetime.datetime.strptime(new_df.iloc[i]['날짜'], "%Y-%m-%d")  # 
        new_df.iloc[i, -4] = date.year
        new_df.iloc[i, -3] = date.month
        new_df.iloc[i, -2] = date.day
        new_df.iloc[i, -1] = weekday_dict[date.weekday()] 
        # weekday()의 결과로 해당 요일에 매칭되는 숫자가 나오고, 이를 dict에서 받아 해당되는 value인 요일명으로 컬럼에 입력
    
    return new_df

## 데이터 분리, 저장

In [12]:
# 지하철 승하차 데이터 불러오기
file_path = "./csvdata/2021_subway_transport_data.csv"

file_df = pd.read_csv(file_path, encoding="EUC-KR", header=0, index_col=0)

,날짜,호선,역번호,역명,구분,06시 이전,06시-07시,07시-08시,08시-09시,09시-10시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시 이후,합 계
2,2021-01-01,1,150,서울역,하차,85,355,438,592,841,...,781,710,626,437,425,354,307,134,77,9951


### 승하차 데이터에서 하차 데이터 분리

In [10]:
# 하차 데이터 분리
arrive_df = initialize_df()  

# 하차 데이터 별도의 데이터프레임에 추가
for i in range(len(file_df)):
    if file_df.iloc[i]['구분'] == '하차':
        target_data = pd.DataFrame(file_df.iloc[i]).T
        arrive_df = pd.concat([arrive_df, target_data], ignore_index=True, axis=0)

# arrive_df.tail()

# 대략 1시간 걸림

,날짜,호선,역번호,역명,구분,06시 이전,06시-07시,07시-08시,08시-09시,09시-10시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시 이후,합 계
102182,2021-12-31,8,2824,단대오거리,하차,54,189,325,639,242,...,714,748,880,1040,737,488,648,402,188,9195
102183,2021-12-31,8,2825,신흥,하차,16,80,122,251,170,...,382,419,444,460,343,223,272,179,91,4683
102184,2021-12-31,8,2826,수진,하차,25,75,96,340,180,...,358,389,407,439,317,223,323,204,100,4605
102185,2021-12-31,8,2827,모란,하차,20,96,120,311,132,...,300,290,286,347,176,107,154,96,52,3402
102186,2021-12-31,8,2828,남위례,하차,10,41,119,375,147,...,213,341,322,382,263,145,209,138,34,3438


In [13]:
# 하차 데이터 저장
# arrive_df.to_csv("./csvdata/2021_arrive_data.csv", encoding="utf-8-sig")

In [ ]:
# 하차 데이터 분리 후, 다시 시작할 때 사용
# arrive_df = pd.read_csv("./csvdata/2021_arrive_data.csv", encoding="utf-8", header=0)

### 분기별 데이터 분리 및 저장

In [ ]:
# 각 분기별 데이터 분리
quarter_01, quarter_02, quarter_03, quarter_04 = quarter_matching(arrive_df)

# 분기별 데이터 저장
# quarter_01.to_csv("./data/2021_arrive_quarter_01.csv", encoding="utf-8-sig")
# quarter_02.to_csv("./data/2021_arrive_quarter_02.csv", encoding="utf-8-sig")
# quarter_03.to_csv("./data/2021_arrive_quarter_03.csv", encoding="utf-8-sig")
# quarter_04.to_csv("./data/2021_arrive_quarter_04.csv", encoding="utf-8-sig")

### 각 분기별 데이터, 시간대 추가하여 정리 및 저장

In [55]:
# 각 분기별 데이터, 시간대별 합산
tz1 = timezone_summation(quarter_01)  # 1 분기
tz2 = timezone_summation(quarter_02)  # 2 분기
tz3 = timezone_summation(quarter_03)  # 3 분기
tz4 = timezone_summation(quarter_04)  # 4 분기

# 각 분기별 데이터 저장
# tz1.to_csv("./data/2021_arrive_qt_tz_01.csv", encoding="utf-8-sig")
# tz2.to_csv("./data/2021_arrive_qt_tz_02.csv", encoding="utf-8-sig")
# tz3.to_csv("./data/2021_arrive_qt_tz_03.csv", encoding="utf-8-sig")
# tz4.to_csv("./data/2021_arrive_qt_tz_04.csv", encoding="utf-8-sig")

### 시간대 추가된 데이터에 연,월,일,요일 추가하여 정리 및 저장

In [56]:
# 분기별 연,월,일,요일 추가
wd1 = ymd_split(tz1)
wd2 = ymd_split(tz2)
wd3 = ymd_split(tz3)
wd4 = ymd_split(tz4)

# 해당 데이터 저장
# wd1.to_csv("./data/2021_arrive_qt_tz_wd_01.csv", encoding="utf-8-sig")
# wd2.to_csv("./data/2021_arrive_qt_tz_wd_02.csv", encoding="utf-8-sig")
# wd3.to_csv("./data/2021_arrive_qt_tz_wd_03.csv", encoding="utf-8-sig")
# wd4.to_csv("./data/2021_arrive_qt_tz_wd_04.csv", encoding="utf-8-sig")

### 필요 데이터 별도 추출 및 저장

In [79]:
# 분기별 데이터에서 필요한 컬럼만 추출
# 역명과 요일을 기준으로 하여, 시간대 데이터를 합산
qt_data = pd.DataFrame(wd1.groupby(['역명','요일'])[['00시11시','11시17시','17시24시']].agg('sum'))

# 해당 행에서, 추가 데이터를 데이터 프레임에 저장
# 역명과 요일에 해당하는 값들을 가지고 데이터를 추가
qt_data['총합'] = qt_data.sum(axis=1)  # 해당 역의 해당 분기+요일의 총합
qt_data['평균'] = round(qt_data[['00시11시','11시17시','17시24시',]].mean(axis=1),0)  # 해당 역의 해당 분기+요일의 평균

# 데이터 저장
# qt_data.to_csv("./data/2021_1qt.csv", encoding='utf-8-sig')